#Setup

In [ ]:
batch_size = 64
epochs = 50
layers = 1
hidden = 32
Dimensions = 5

In [ ]:
import math
import random

import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.nn.parameter import Parameter, UninitializedParameter
import torch.nn.functional as F
from torch.nn.modules.module import Module
from torch.nn import init
from tqdm.notebook import trange, tqdm
import torchvision.transforms.functional as TF
from typing import Optional, List, Tuple, Union
from torch.nn.common_types import _size_1_t, _size_2_t, _size_3_t
import itertools
import numpy as np

In [ ]:
# Download training data from open datasets.
training_data = datasets.MNIST(root="data", train=True, download=True, 
                                      transform=transforms.Compose([transforms.Resize((16, 16)), 
                                                                    transforms.Pad(8), 
                                                                    transforms.ToTensor()]),)

# Download test data from open datasets.
test_data = datasets.MNIST(root="data", train=False, download=True,
                                  transform=transforms.Compose([transforms.Resize((16, 16)), 
                                                                    transforms.Pad(8), 
                                                                    transforms.ToTensor()]),)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
# Create data loaders.
train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

for X, y in test_loader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Shape of X [N, C, H, W]:  torch.Size([64, 1, 32, 32])
Shape of y:  torch.Size([64]) torch.int64


# MLP with search in alpha-space



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def count_model_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class HHN_MLPB(nn.Module):
    def __init__(self, hin, dimensions, n_layers, n_units, n_channels, n_classes=10):
        super(HHN_MLPB, self).__init__()
        self.hyper_stack = nn.Sequential(
            nn.Linear(hin, 64),
            nn.ReLU(),
            nn.Linear(64, dimensions),
            nn.Softmax(dim=0)
        )

        self.dimensions = dimensions
        self.n_layers = n_layers
        self.n_units = n_units
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        self.running_mu = torch.zeros(self.n_units).to(self.device)  # zeros are fine for first training iter
        self.running_std = torch.ones(self.n_units).to(self.device)  # ones are fine for first training iter

        self.weight_list_fc1, self.bias_list_fc1 = \
            self.create_param_combination_linear(in_features=32 * 32 * n_channels, out_features=n_units)
        self.weights = nn.ParameterList()
        self.biases = nn.ParameterList()
        for _ in range(n_layers - 1):
            w, b = self.create_param_combination_linear(in_features=n_units, out_features=n_units)
            self.weights += w
            self.biases += b
        self.weight_list_fc2, self.bias_list_fc2 = self.create_param_combination_linear(in_features=n_units,
                                                                                        out_features=n_classes)

    def create_param_combination_linear(self, in_features, out_features):
        weight_list = nn.ParameterList()
        bias_list = nn.ParameterList()
        for _ in range(self.dimensions):
            weight = Parameter(torch.empty((out_features, in_features)))
            nn.init.kaiming_uniform_(weight, a=math.sqrt(5))
            weight_list.append(weight)

            bias = Parameter(torch.empty(out_features))
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(weight)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(bias, -bound, bound)
            bias_list.append(bias)
        return weight_list, bias_list

    def calculate_weighted_sum(self, param_list: List, factors: Tensor):
        weighted_list = [a * b for a, b in zip(param_list, factors)]
        return torch.sum(torch.stack(weighted_list), dim=0)

    def forward(self, x, hyper_x):
        hyper_output = self.hyper_stack(hyper_x)

        weight_fc1 = self.calculate_weighted_sum(self.weight_list_fc1, hyper_output)
        weight_fc2 = self.calculate_weighted_sum(self.weight_list_fc2, hyper_output)

        bias_fc1 = self.calculate_weighted_sum(self.bias_list_fc1, hyper_output)
        bias_fc2 = self.calculate_weighted_sum(self.bias_list_fc2, hyper_output)

        logits = torch.flatten(x, start_dim=1)
        logits = F.linear(logits, weight=weight_fc1, bias=bias_fc1)
        logits = torch.relu(logits)

        it_w = iter(self.weights)
        it_b = iter(self.biases)
        for (w, b) in zip(zip(*[it_w] * self.dimensions), zip(*[it_b] * self.dimensions)):
            w = nn.ParameterList(w)
            b = nn.ParameterList(b)
            w = self.calculate_weighted_sum(w.to(self.device), hyper_output)
            b = self.calculate_weighted_sum(b.to(self.device), hyper_output)
            logits = F.linear(logits, weight=w, bias=b)
            logits = F.batch_norm(logits, self.running_mu, self.running_std, training=True, momentum=0.9)
            logits = torch.relu(logits)
        logits = F.linear(logits, weight=weight_fc2, bias=bias_fc2)
        return logits

model = HHN_MLPB(2, Dimensions, layers, hidden, 1, n_classes=10).to(device)
print(model)
print(count_model_parameters(model))

HHN_MLPB(
  (hyper_stack): Sequential(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=5, bias=True)
    (3): Softmax(dim=0)
  )
  (weight_list_fc1): ParameterList(
      (0): Parameter containing: [torch.float32 of size 32x1024 (GPU 0)]
      (1): Parameter containing: [torch.float32 of size 32x1024 (GPU 0)]
      (2): Parameter containing: [torch.float32 of size 32x1024 (GPU 0)]
      (3): Parameter containing: [torch.float32 of size 32x1024 (GPU 0)]
      (4): Parameter containing: [torch.float32 of size 32x1024 (GPU 0)]
  )
  (bias_list_fc1): ParameterList(
      (0): Parameter containing: [torch.float32 of size 32 (GPU 0)]
      (1): Parameter containing: [torch.float32 of size 32 (GPU 0)]
      (2): Parameter containing: [torch.float32 of size 32 (GPU 0)]
      (3): Parameter containing: [torch.float32 of size 32 (GPU 0)]
      (4): Parameter containing: [torch.float32 of size 32 (GPU 0)]
  )
  (weights): Par

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs, eta_min=0)

cos = nn.CosineSimilarity(dim=0, eps=1e-6)

def train(dataloader, model, loss_fn, optimizer):
    for batch, (X, y) in enumerate(tqdm(dataloader, desc='Training')):
        X, y = X.to(device), y.to(device)
        a, b = random.uniform(-8, 8), random.uniform(-8, 8)
        X = TF.affine(X, scale=1.0, angle=0, translate=(a, b), shear=0.0)

        pred = model(X, hyper_x=Tensor([a, b]).to(device))
        loss = loss_fn(pred, y)

        beta1 = model.hyper_stack(Tensor([a, b]).to(device))
        a2, b2 = random.uniform(-8, 8), random.uniform(-8, 8)
        beta2 = model.hyper_stack(Tensor([a2, b2]).to(device))
        loss += pow(cos(beta1, beta2),2)

        # minimize entropy to the correct degree
        b = (F.softmax(pred, dim=1)) * (-1 * F.log_softmax(pred, dim=1))
        loss += 0.01*b.sum()

        # maximize entropy to the wrong degree
        logits = model(X, hyper_x=Tensor([a2, b2]).to(device))
        b2 = (F.softmax(logits, dim=1)) * (-1 * F.log_softmax(logits, dim=1))
        loss -= 0.01*b2.sum()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # scheduler.step()

def validate(dataloader, model, loss_fn):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            a, b = random.uniform(-8, 8), random.uniform(-8, 8)
            X = TF.affine(X, scale=1.0, angle=0, translate=(a, b), shear=0.0)

            pred = model(X, hyper_x=Tensor([a, b]).to(device))
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    print(f"Test with translation={a,b}: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    return correct, test_loss

for t in range(epochs):
    print(f"=================\n Epoch: {t + 1} \n=================")
    train(train_loader, model, loss_fn, optimizer)
    test_acc, test_loss = validate(test_loader, model, loss_fn)
print("Done!")

 Epoch: 1 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

Test with translation=(2.3260463607721835, 2.6758056233708736): Accuracy: 40.9%, Avg loss: 1.728002
 Epoch: 2 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

Test with translation=(0.9998305250234889, 3.7428690667227436): Accuracy: 53.1%, Avg loss: 1.369338
 Epoch: 3 


...

Test with translation=(-6.101546567503137, 3.5178696881288545): Accuracy: 85.5%, Avg loss: 0.472860
 Epoch: 49 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

Test with translation=(-2.9041162097834654, 5.802479612075638): Accuracy: 85.1%, Avg loss: 0.485140
 Epoch: 50 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

Test with translation=(-7.7357973748913444, 7.989186992718732): Accuracy: 85.4%, Avg loss: 0.497362
Done!


Optimize in the alpha space

In [ ]:
# model to eval mode and move to cpu
model.eval()
model.cpu()

# freeze Ws
for param in model.parameters():
  param.requires_grad = False

In [ ]:
# execute only if you wish to test with a different batch_size (for example, batch_size=1 ... takes long!)
batch_size=64
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
from scipy import optimize

# function to minimize by the basin hopping algorithm
def f(z, *args):
    a, b = ((z + 1)*8)%16 - 8
    X = args[0]
    logits = model(Tensor(X), hyper_x=Tensor([a, b]))
    b = (F.softmax(logits, dim=1)) * (-1 * F.log_softmax(logits, dim=1))  # entropy
    return b.sum().numpy()

# given a batch of images find the rotation angle alpha
def findalpha(X):
    # Basin hopping algorithm
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.basinhopping.html
    minimizer_kwargs = {"method": "BFGS", "args":X}
    res = optimize.basinhopping(f, [0.0, 0.0], minimizer_kwargs=minimizer_kwargs, niter=100, T=50)

    alpha = ((res.x + 1)*8)%16 - 8
    # print("alpha estimate = ", alpha)     # obtained minimum
    # print("fun = ", res.fun)              # function value at minimum
    return alpha

result = 0.0
for _, (X, y) in enumerate(tqdm(test_loader, desc='Testing alpha search')):
    a, b = random.uniform(-8, 8), random.uniform(-8, 8)
    # print("=============")
    # print("alpha true = ", [a,b])
    X = TF.affine(X, scale=1.0, angle=0, translate=(a, b), shear=0.0)

    alpha = findalpha(X)

    # compute model prediction with the estimated alpha
    logits = model(X, hyper_x=Tensor(alpha))
    # y is the true label --> calculate accuracy
    correct = (logits.argmax(1) == y).type(torch.float).sum().item() / batch_size
    # print(f"accuracy = {(100*correct):>0.1f}")
    result += correct

result /= len(test_loader.dataset) / batch_size
print(f"Test accuracy: {(100*result):>0.1f}%")

print("Done!")

Testing alpha search:   0%|          | 0/157 [00:00<?, ?it/s]

Test accuracy: 84.4%
Done!


# One4All

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MLPB(nn.Module):
    def __init__(self, n_layers, n_units, n_channels, n_classes=10):
        super(MLPB, self).__init__()
        mid_layers = []
        mid_layers.extend([nn.Flatten(), nn.Linear(32 * 32 * n_channels, n_units), nn.ReLU()])
        for _ in range(n_layers-1):
            mid_layers.extend([
                nn.Linear(n_units, n_units),
                nn.BatchNorm1d(n_units, momentum=0.9),
                nn.ReLU(),
            ])
        mid_layers.extend([nn.Linear(n_units, n_classes)])
        self.linear_relu_stack = nn.Sequential(*mid_layers)

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = MLPB(layers, hidden, 1, n_classes=10).to(device)
print(model)
print(count_model_parameters(model))

MLPB(
  (linear_relu_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1024, out_features=32, bias=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=10, bias=True)
  )
)
33130


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs, eta_min=0)

def train(dataloader, model, loss_fn, optimizer):
    for batch, (X, y) in enumerate(tqdm(dataloader, desc='Training')):
        X, y = X.to(device), y.to(device)
        a, b = random.uniform(-8, 8), random.uniform(-8, 8)
        X = TF.affine(X, scale=1.0, angle=0, translate=(a, b), shear=0.0)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # scheduler.step()

def validate(dataloader, model, loss_fn):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            a, b = random.uniform(-8, 8), random.uniform(-8, 8)
            X = TF.affine(X, scale=1.0, angle=0, translate=(a, b), shear=0.0)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    print(f"Test with translation={a,b}: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    return correct, test_loss

for t in range(epochs):
    print(f"=================\n Epoch: {t + 1} \n=================")
    train(train_loader, model, loss_fn, optimizer)
    test_acc, test_loss = validate(test_loader, model, loss_fn)
print("Done!")

 Epoch: 1 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test with translation=(2.220755577066466, 4.929664021916047): Accuracy: 39.5%, Avg loss: 1.752685
 Epoch: 2 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

...

Test with translation=(0.4947995121703048, -3.204277199042691): Accuracy: 79.8%, Avg loss: 0.652544
 Epoch: 49 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

Test with translation=(-5.0801060276455825, -3.3254813299776345): Accuracy: 80.2%, Avg loss: 0.651436
 Epoch: 50 


Training:   0%|          | 0/938 [00:00<?, ?it/s]

Test with translation=(-4.570527886771533, -4.845838056985331): Accuracy: 79.4%, Avg loss: 0.649829
Done!
